## Accessibility Analysis for Bangladesh using GOSTnets Raster
Generates population weighted shorterst travel time to ports
### Inputs:
    - origins: population grid
    - destinations: Ports
    
Friction layer is the Global Friction Surface 2019 from the Malaria Access Project (https://malariaatlas.org/research-project/accessibility-to-healthcare/)

In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
sys.path.append(r"C:\repos\GOSTnets")
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features

from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

# sys.path.append(r"C:\repos\INFRA_SAP")
# import infrasap.market_access as ma
# importlib.reload(ma)

C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
sys.path.append(r"C:\repos\gostrocks\src")
import GOSTRocks.rasterMisc as rMisc
importlib.reload(rMisc)

<module 'GOSTRocks.rasterMisc' from 'C:\\repos\\gostrocks\\src\\GOSTRocks\\rasterMisc.py'>

In [3]:
sys.path.append(r"C:\repos\GOSTNets_Raster\src")
import GOSTNetsRaster.market_access as ma
importlib.reload(ma)

<module 'GOSTNetsRaster.market_access' from 'C:\\repos\\GOSTNets_Raster\\src\\GOSTNetsRaster\\market_access.py'>

In [4]:
global_friction = r"D:\data\global_friction_surface\2020_motorized_friction_surface.geotiff"

In [5]:
origins_file = r"inputs\bgd_pd_2020_1km.tif"

In [6]:
port_facilities = r"inputs\BGD_ports_Chittagong.shp"
port_facilities = gpd.read_file(port_facilities)

In [7]:
# You need the bounding box to be a min bounding box, not the actual shape or else the results are weird 
admin = r"inputs\BGD_corridor1_min_bound.shp"

In [8]:
out_folder = r"outputs"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [9]:
friction_file = os.path.join(out_folder, "friction_surface.tif")

In [10]:
#clip global friction surface to admin area
if not os.path.exists(friction_file):
    rMisc.clipRaster(rasterio.open(global_friction), gpd.read_file(admin), friction_file)

## Standardize or Co-register population file to match the friction surface raster

In [11]:
# create friction surface
inR = rasterio.open(friction_file)

In [12]:
inPop = rasterio.open(origins_file)
# Make sure that both rasters have the exact same resolution, crs, and number of pixels
out_pop_surface_std = os.path.join(out_folder, "BGD_pd_2020_1km_STD.tif")
if not os.path.exists(out_pop_surface_std):
    rMisc.standardizeInputRasters(inPop, inR, out_pop_surface_std, data_type="C")

## create friction surface

The actual units within the friction surface raster are minutes required to travel one meter. Therefore multiple by 1000 to get an approximate time in minutes it takes to cross a pixel, because 30-arcsec resolution pixel is approx. 1km by 1km at the equator. However, Bangladesh is about 23.5 degrees north of the equator.

At the equator, an arc-second of longitude approximately equals 30.87 meters. Arc-seconds of longitude decrease in a trigonometric cosine-based fashion as one moves toward the earth's poles (https://www.esri.com/news/arcuser/0400/wdside.html).

In [1]:
import math

In [2]:
# COS of 23.5 degrees * length of arc-sec at equator * 30-arcsec
math.cos(math.radians(23.5)) * 30.87 * 30

849.2893348880634

In [13]:
frictionD = inR.read()[0,:,:] * 849
mcp = graph.MCP_Geometric(frictionD)

In [14]:
type(frictionD)

numpy.ndarray

In [15]:
inR.crs

CRS.from_epsg(4326)

## Calculate Travel Time

In [16]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, port_facilities, out_raster=os.path.join(out_folder, "BGD_least_cost_travel_time_ports.tif"))

### Clip results to corridor admin

In [17]:
corridor_admin = r"inputs\BGD_corridor1_dissolved.shp"

In [18]:
clipped_file_ports = os.path.join(out_folder, "BGD_least_cost_travel_time_ports_clipped.tif")
if not os.path.exists(clipped_file_ports):
    rMisc.clipRaster(rasterio.open(os.path.join(out_folder, "BGD_least_cost_travel_time_ports.tif")), gpd.read_file(corridor_admin), clipped_file_ports)

## Do Zonal Stats: Population weighted average

In [19]:
# do zonal stats (pop weighted average) as found here: https://github.com/worldbank/GOST_PublicGoods/blob/master/Implementations/FY21/URB_TurkeyUrbanizationReview/URB_TurkeyUrbanizationReview.ipynb

In [20]:
ttR_ports = rasterio.open(os.path.join(out_folder, "BGD_least_cost_travel_time_ports.tif"))
ttD_ports = ttR_ports.read()

popR = rasterio.open(os.path.join(out_folder, "BGD_pd_2020_1km_STD.tif"))
popD = popR.read()

In [21]:
ttD_ports.shape

(1, 424, 282)

In [22]:
popD.shape

(1, 424, 282)

In [23]:
ttD_ports

array([[[         inf,          inf,          inf, ...,          inf,
                  inf,          inf],
        [         inf, 368.35214739, 368.89580269, ..., 393.46276917,
         416.32432999, 403.88265542],
        [         inf, 365.72714739, 366.27080269, ..., 392.74630617,
         396.26702375, 392.77613048],
        ...,
        [         inf, 780.5300421 , 779.38474155, ..., 262.66336934,
         262.1032396 , 263.38108271],
        [         inf, 783.2950422 , 782.14974166, ..., 264.41336946,
         264.29073966, 265.29582688],
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf]]])

In [24]:
popD

array([[[  1056.4373 ,   1068.2314 ,    902.06525, ..., -99999.     ,
         -99999.     , -99999.     ],
        [  1098.9989 ,   1035.6807 ,    833.0178 , ..., -99999.     ,
         -99999.     , -99999.     ],
        [  1171.2362 ,    942.9522 ,    862.8866 , ..., -99999.     ,
         -99999.     , -99999.     ],
        ...,
        [-99999.     , -99999.     , -99999.     , ...,   3087.4536 ,
           1950.1735 ,    773.5452 ],
        [-99999.     , -99999.     , -99999.     , ...,   1131.2036 ,
           1072.3378 ,    655.13727],
        [-99999.     , -99999.     , -99999.     , ...,    383.39288,
            398.62497,    446.42493]]], dtype=float32)

In [25]:
# code to replace all negative value with 0
popD[popD<0] = 0

In [26]:
popD

array([[[1056.4373 , 1068.2314 ,  902.06525, ...,    0.     ,
            0.     ,    0.     ],
        [1098.9989 , 1035.6807 ,  833.0178 , ...,    0.     ,
            0.     ,    0.     ],
        [1171.2362 ,  942.9522 ,  862.8866 , ...,    0.     ,
            0.     ,    0.     ],
        ...,
        [   0.     ,    0.     ,    0.     , ..., 3087.4536 ,
         1950.1735 ,  773.5452 ],
        [   0.     ,    0.     ,    0.     , ..., 1131.2036 ,
         1072.3378 ,  655.13727],
        [   0.     ,    0.     ,    0.     , ...,  383.39288,
          398.62497,  446.42493]]], dtype=float32)

In [27]:
ttPop_ports = popD * ttD_ports

<ipython-input-27-ca5e440e7b48>:1: RuntimeWarning: invalid value encountered in multiply
  ttPop_ports = popD * ttD_ports


In [28]:
ttPop_ports

array([[[            inf,             inf,             inf, ...,
                     nan,             nan,             nan],
        [            inf, 381495.19661262, 307296.77819616, ...,
              0.        ,      0.        ,      0.        ],
        [            inf, 344863.22169089, 316050.16639259, ...,
              0.        ,      0.        ,      0.        ],
        ...,
        [            nan,      0.        ,      0.        , ...,
         810960.96873656, 511146.78214924, 203737.17942573],
        [            nan,      0.        ,      0.        , ...,
         299105.35892889, 283408.94201604, 173805.18324852],
        [            nan,             nan,             nan, ...,
                     inf,             inf,             inf]]])

In [29]:
out_meta = ttR_ports.meta.copy()
with rasterio.open(os.path.join(out_folder, "ttPop_ports_raster.tif"), "w", **out_meta) as dest:
    dest.write(ttPop_ports.astype(out_meta['dtype']))

In [30]:
# save pop raster too for later
out_meta = ttR_ports.meta.copy()
with rasterio.open(os.path.join(out_folder, "Pop_raster.tif"), "w", **out_meta) as dest:
    dest.write(popD.astype(out_meta['dtype']))

In [31]:
#rasterStats, ignore negative values
from rasterstats import zonal_stats

In [32]:
admin1 = r"inputs\BGD_corridor1_dissolved.shp"

In [33]:
ttPop_ports_raster = r"outputs\ttPop_ports_raster.tif"

In [34]:
#stats = zonal_stats(admin1, ttPop_raster, geojson_out=True)

### consider using an admin or an urban exent file here for better summaries

In [35]:
BGD_adm1_df = gpd.GeoDataFrame.from_file(r"inputs\BGD_corridor1_dissolved.shp")

In [36]:
ttPop_ports_sum_stats = zonal_stats(admin1, ttPop_ports_raster, stats='sum')

In [37]:
#pd.DataFrame(ttPop_HF_sum_stats)

In [38]:
ttPop_ports_sum_stats_joined = BGD_adm1_df.join(pd.DataFrame(ttPop_ports_sum_stats))

In [39]:
#ttPop_HF_sum_stats_joined

In [40]:
# Now need to do rasterstats for just pop, then divide the sum of each of the ttPop zones with the sum pop for each zone

In [41]:
Pop_raster = r"outputs\Pop_raster.tif"

In [42]:
statsPop = zonal_stats(admin1, Pop_raster, stats=['sum'])

In [43]:
statsPop_df = pd.DataFrame(statsPop)

In [44]:
statsPop_df.rename(columns={'sum':'popsum'}, inplace=True)

In [45]:
statsPop_df

,popsum
0,5.307779e+07


In [46]:
ttPop_ports_sum_stats_joined2 = ttPop_ports_sum_stats_joined.join(pd.DataFrame(statsPop_df))

In [47]:
ttPop_ports_sum_stats_joined2

,F_Opacity,F_Componen,F_LayerNam,F_Id,F_Name,F_AreaDire,F_VertexCo,F_Style,F_StrokeWe,F_Perimete,...,ISO,NAME_0,ID_1,NAME_1,Shape_Leng,Shape_Area,NAME_2,geometry,sum,popsum
0,0,0,None,0,None,0,0,None,0.0,0.0,...,BGD,Bangladesh,0,Chittagong,0.628114,0.007252,Brahmanbaria,"MULTIPOLYGON (((92.33435 20.75475, 92.33490 20...",1.058177e+10,5.307779e+07


In [48]:
ttPop_ports_sum_stats_joined2['avg_tt'] = ttPop_ports_sum_stats_joined2['sum'] / ttPop_ports_sum_stats_joined2['popsum']

In [49]:
ttPop_ports_sum_stats_joined2

,F_Opacity,F_Componen,F_LayerNam,F_Id,F_Name,F_AreaDire,F_VertexCo,F_Style,F_StrokeWe,F_Perimete,...,NAME_0,ID_1,NAME_1,Shape_Leng,Shape_Area,NAME_2,geometry,sum,popsum,avg_tt
0,0,0,None,0,None,0,0,None,0.0,0.0,...,Bangladesh,0,Chittagong,0.628114,0.007252,Brahmanbaria,"MULTIPOLYGON (((92.33435 20.75475, 92.33490 20...",1.058177e+10,5.307779e+07,199.363417


In [50]:
ttPop_ports_sum_stats_joined2.to_file("ttPop_ports_sum_stats_joined2.shp")

In [51]:
# Calculate Market Access
#https://github.com/worldbank/GOST_PublicGoods/blob/master/Implementations/FY21/ACC_Raster_MarketAccess_Template/ACC_Benin_RasterMarketAccess.ipynb